# Data project: Price cycles in the market for prescription drugs after patent expiry

## Content
### 1. Introduction
### 2. Data reading, cleaning and selecting
### 3. Expectations to the market mechanisms (normal situation)
### 4. Analyzing the drug Lamotrigin (price cycles)
### 5. Coonclusion


## 1. Introduction

In this project we will attempt to show the cyclic behaviour in some medical prices after the patent has expired, observed by the study "Priscykler i markedet for receptpligtig medicin efter patentudløb”. It will serve to demonstrate the matter of possible colusion by involving key points from competition theory by Maskin and Tirole (1988). In the empirical analysis we will investigate whether these cycles appears in data for the danish medicine prices in the period 2nd febuary 2015 - 23rd of marts 2020. 

Pharmaceutical manufacturers in Denmark sell medicines through two channels: the primary healthcare sector, which involves sales to the pharmacies, and the secondary sector in which sales are made to
hospitals. 

Before the competing phase, the company that developed the drug has a patent. The patent creates an incentive for the company to invest in new medicines, and with the exclusive right to sell the drug for a number of years, the company can receive research and production costs. In the period when the patent is valid, the market can be characterized as monopolistic and may have an inelastic demand curve, depending on of the type of drug. 

After the expiry of the patent, the prices of drugs that are sold to the pharmacies directly by the competing companies. Every 14 days, companies have the opportunity to report a new price electronically, where the company has
has bid the lowest price in the same substitution group, wins the bidding round. The pharmacists are obliged to offer patients the cheapest drug within the same substitute group,
and thus the company secures the largest market share, approx. 70 per cent until the next bidding round (”Priscykler i markedet for receptpligtig medicin efter patentudløb” p. 2). 

The data we will be working with in this project is containing these exactly 14-days prices.

## 2. Data reading, cleaning and selecting

Import and set magics: 

In [208]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import ipywidgets as widgets
import seaborn as sns; sns.set()
import dataproject

In [209]:
#1. Importing excel sheet for analysis
medprices_dk = pd.read_excel('meddata.xlsx')
medprices_dk

,ATC,Lægemiddel,Varenummer,Pakning,Styrke,Form,Firma,Indikator,20150202,20150216,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
0,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AIP,407.36,407.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP,570.25,570.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP_pr_DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A01AA01,Bifluorid,43158,10 g,NaN,dentalsuspension,Voco,AIP,602.07,602.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64863,V08DA05,SonoVue,5615,1 sæt,8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Bracco Imaging Scandinavia,AUP_pr_DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64864,V08DA05,SonoVue,589726,1 sæt (Orifarm),8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Orifarm,AIP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64865,V08DA05,SonoVue,589726,1 sæt (Orifarm),8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Orifarm,AUP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64866,V08DA05,SonoVue,589726,1 sæt (Orifarm),8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Orifarm,DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
# 2. Editing frame structure

english = {'Lægemiddel':'Medicine', 
                  'Varenummer':'Product_number', 
                  'Pakning':'Packaging', 
                  'Styrke':'Strength', 
                  'Firma':'Company', 
                  'Indikator':'Indicator'}
medprices_dk.rename(columns = english, inplace=True) # Convert columns from Danish to English

column_v = list(medprices_dk.columns.values) # Create list of dataframes column names
column_v= column_v[8:]# narrow list to only focus on date variables

date_to_string = {} # Create dictionary for renaming date variables

# Loop throught all possible date variables and add an e in front as variables in python cannot begin with a numbers
for i in column_v:
    date_to_string[i] = f'e{i}'

medprices_dk.rename(columns = date_to_string, inplace=True) # Rename date variables in primary dataframe

medprices_dk



,ATC,Medicine,Product_number,Packaging,Strength,Form,Company,Indicator,e20150202,e20150216,...,e20191118,e20191202,e20191216,e20191230,e20200113,e20200127,e20200210,e20200224,e20200309,e20200323
0,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AIP,407.36,407.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP,570.25,570.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP_pr_DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A01AA01,Bifluorid,43158,10 g,NaN,dentalsuspension,Voco,AIP,602.07,602.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64863,V08DA05,SonoVue,5615,1 sæt,8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Bracco Imaging Scandinavia,AUP_pr_DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64864,V08DA05,SonoVue,589726,1 sæt (Orifarm),8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Orifarm,AIP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64865,V08DA05,SonoVue,589726,1 sæt (Orifarm),8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Orifarm,AUP,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64866,V08DA05,SonoVue,589726,1 sæt (Orifarm),8 mikroliter/ml,"pulver og solvens til injektionsvæske, disperg...",Orifarm,DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Expectations to the market mechanisms (normal situation)

According to theory, the transition from a monopolistic market state to complete competition should push down the prices of medicines, as several players can enter the market. As the price moves towards marginal costs, fiercer competition arises. Of course, there may be supply or demand shocks, costly entry barriers, etc., but all else equal, the price should fall after patent expiry. 

In order to illustrate this, we have found a drug with an expiry of the patent in april 2015. 
The drug is named Aripiprazole, and is a treatment of psychoses. At the beginning of 2015,full-year forecasts were expected to see a major fall in prices for the drug immediately following the expiry of the patent, ie April 2015. However, the price falls did not break through until the end of 2015. Price falls are now at 80-90%.

INDSÆT GRAPH MED PRISUDVIKLINGEN 

In [211]:
medprices_dk_copy2 = medprices_dk.copy()
aripiprazol2=medprices_dk_copy2.loc[medprices_dk_copy2['Medicine'].isin(['Aripiprazol "Accord Healthcare"', 'Aripiprazol "Krka"','Aripiprazol "Stada"','Aripiprazol "Accord"','Aripiprazol "Aristo"','Aripiprazol "Medical Valley"','Aripiprazol "Sandoz"','Aripiprazol "Teva"']) ]
a=aripiprazol2.Indicator.str.contains('AUP_pr_DDD')
aripiprazol2=aripiprazol2.loc[a, :]
drop_list=['ATC', 'Form', 'Medicine']
aripiprazol2.drop(drop_list, axis=1, inplace=True)
aripiprazol2.reset_index()

,index,Product_number,Packaging,Strength,Company,Indicator,e20150202,e20150216,e20150302,e20150316,...,e20191118,e20191202,e20191216,e20191230,e20200113,e20200127,e20200210,e20200224,e20200309,e20200323
0,46883,79410,100 stk.,10 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,5.579972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,46887,113971,28 stk. (blister),10 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46891,173895,100 stk.,15 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,0.455000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46895,178885,150 ml,1 mg/ml,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46899,418119,56 stk. (blister),5 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,46903,506809,100 stk.,5 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,3.816038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,46907,540648,56 stk. (blister),15 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,46911,542634,56 stk. (blister),10 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,46915,554817,28 stk. (blister),15 mg,Teva (S›borg),AUP_pr_DDD,NaN,NaN,NaN,NaN,...,0.792857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,46919,52737,56 stk. (blister),5 mg,Krka AB,AUP_pr_DDD,119.304120,112.714952,98.810736,73.193336,...,1.995500,1.757111,1.757111,1.518723,1.513366,1.513366,1.513366,1.513366,1.384797,1.384797


In [212]:
# 4. Create long format of dataframe 

aripiprazol_long= pd.wide_to_long(aripiprazol2, stubnames='e', i="Product_number", j='Date') # Created long format of the lamotrigin Dataframe
e_grouped = aripiprazol_long.groupby('Date')['e'] # Grouped by product number
aripiprazol_long= aripiprazol_long.sort_values(['Date','Product_number']) # Organized product number by date
aripiprazol_long

,,Indicator,Packaging,Strength,Company,e
Product_number,Date,,,,,
37768,20150202,AUP_pr_DDD,28 stk. (blister),5 mg,PharmaCoDane,NaN
52737,20150202,AUP_pr_DDD,56 stk. (blister),5 mg,Krka AB,119.30412
63519,20150202,AUP_pr_DDD,150 ml,1 mg/ml,Stada Nordic,NaN
72209,20150202,AUP_pr_DDD,28 stk. (blister),15 mg,PharmaCoDane,NaN
79410,20150202,AUP_pr_DDD,100 stk.,10 mg,Teva (S›borg),NaN
...,...,...,...,...,...,...
521910,20200323,AUP_pr_DDD,28 stk. (blister),30 mg,Krka AB,NaN
523039,20200323,AUP_pr_DDD,28 stk. (blister),10 mg,PharmaCoDane,NaN
540648,20200323,AUP_pr_DDD,56 stk. (blister),15 mg,Teva (S›borg),NaN


In [213]:

aripiprazol_long.isnull().sum() # Explore how many null values are in each column of your dataset


Indicator       0
Packaging       0
Strength        0
Company         0
e            1623
dtype: int64

In [214]:
aripiprazol_long=aripiprazol_long[aripiprazol_long['e'].notna()]
aripiprazol_long

,,Indicator,Packaging,Strength,Company,e
Product_number,Date,,,,,
52737,20150202,AUP_pr_DDD,56 stk. (blister),5 mg,Krka AB,119.304120
88445,20150202,AUP_pr_DDD,56 stk. (blister),15 mg,Krka AB,29.277679
107657,20150202,AUP_pr_DDD,56 stk. (blister),10 mg,Krka AB,57.091581
113143,20150202,AUP_pr_DDD,56 stk. (blister),30 mg,Krka AB,33.058929
114662,20150202,AUP_pr_DDD,14 stk. (blister),5 mg,Krka AB,140.529248
...,...,...,...,...,...,...
396362,20200323,AUP_pr_DDD,56 stk. (blister),10 mg,PharmaCoDane,2.025018
403579,20200323,AUP_pr_DDD,28 stk. (blister),15 mg,PharmaCoDane,21.775000
427375,20200323,AUP_pr_DDD,30 stk. (blister),15 mg,PharmaCoDane,1.261667


In [215]:
aripiprazol_long.isnull().sum() # Explore how many null values are in each column of your dataset

Indicator    0
Packaging    0
Strength     0
Company      0
e            0
dtype: int64

In [216]:
aripiprazol_long.rename(columns={"e": "Price"}, inplace=True)
aripiprazol_long

,,Indicator,Packaging,Strength,Company,Price
Product_number,Date,,,,,
52737,20150202,AUP_pr_DDD,56 stk. (blister),5 mg,Krka AB,119.304120
88445,20150202,AUP_pr_DDD,56 stk. (blister),15 mg,Krka AB,29.277679
107657,20150202,AUP_pr_DDD,56 stk. (blister),10 mg,Krka AB,57.091581
113143,20150202,AUP_pr_DDD,56 stk. (blister),30 mg,Krka AB,33.058929
114662,20150202,AUP_pr_DDD,14 stk. (blister),5 mg,Krka AB,140.529248
...,...,...,...,...,...,...
396362,20200323,AUP_pr_DDD,56 stk. (blister),10 mg,PharmaCoDane,2.025018
403579,20200323,AUP_pr_DDD,28 stk. (blister),15 mg,PharmaCoDane,21.775000
427375,20200323,AUP_pr_DDD,30 stk. (blister),15 mg,PharmaCoDane,1.261667


In [217]:
aripiprazol_long.loc[aripiprazol_long['Packaging'] == '56 stk. (blister)']

,,Indicator,Packaging,Strength,Company,Price
Product_number,Date,,,,,
52737,20150202,AUP_pr_DDD,56 stk. (blister),5 mg,Krka AB,119.304120
88445,20150202,AUP_pr_DDD,56 stk. (blister),15 mg,Krka AB,29.277679
107657,20150202,AUP_pr_DDD,56 stk. (blister),10 mg,Krka AB,57.091581
113143,20150202,AUP_pr_DDD,56 stk. (blister),30 mg,Krka AB,33.058929
52737,20150216,AUP_pr_DDD,56 stk. (blister),5 mg,Krka AB,112.714952
...,...,...,...,...,...,...
107657,20200323,AUP_pr_DDD,56 stk. (blister),10 mg,Krka AB,2.350467
113143,20200323,AUP_pr_DDD,56 stk. (blister),30 mg,Krka AB,0.723214
197123,20200323,AUP_pr_DDD,56 stk. (blister),30 mg,PharmaCoDane,0.723214


In [218]:
aripiprazol_long['Price'].max()

141.793443325477

In [219]:
aripiprazol_long['Price'].min()

0.113392857142857

In [220]:
del aripiprazol_long['Indicator']
aripiprazol_long

,,Packaging,Strength,Company,Price
Product_number,Date,,,,
52737,20150202,56 stk. (blister),5 mg,Krka AB,119.304120
88445,20150202,56 stk. (blister),15 mg,Krka AB,29.277679
107657,20150202,56 stk. (blister),10 mg,Krka AB,57.091581
113143,20150202,56 stk. (blister),30 mg,Krka AB,33.058929
114662,20150202,14 stk. (blister),5 mg,Krka AB,140.529248
...,...,...,...,...,...
396362,20200323,56 stk. (blister),10 mg,PharmaCoDane,2.025018
403579,20200323,28 stk. (blister),15 mg,PharmaCoDane,21.775000
427375,20200323,30 stk. (blister),15 mg,PharmaCoDane,1.261667


In [221]:

del aripiprazol_long['Packaging']
#del aripiprazol_long['Strength']
aripiprazol_long

,,Strength,Company,Price
Product_number,Date,,,
52737,20150202,5 mg,Krka AB,119.304120
88445,20150202,15 mg,Krka AB,29.277679
107657,20150202,10 mg,Krka AB,57.091581
113143,20150202,30 mg,Krka AB,33.058929
114662,20150202,5 mg,Krka AB,140.529248
...,...,...,...,...
396362,20200323,10 mg,PharmaCoDane,2.025018
403579,20200323,15 mg,PharmaCoDane,21.775000
427375,20200323,15 mg,PharmaCoDane,1.261667


In [ ]:
## 4. Analyzing the drug Lamotrigin (price cycles)

In [222]:
del aripiprazol_long['Strength']
aripiprazol_long

,,Company,Price
Product_number,Date,,
52737,20150202,Krka AB,119.304120
88445,20150202,Krka AB,29.277679
107657,20150202,Krka AB,57.091581
113143,20150202,Krka AB,33.058929
114662,20150202,Krka AB,140.529248
...,...,...,...
396362,20200323,PharmaCoDane,2.025018
403579,20200323,PharmaCoDane,21.775000
427375,20200323,PharmaCoDane,1.261667


In [237]:
e.min = aripiprazol_long.groupby('Date').min()
e.min


,Company,Price
Date,,
20150202,Krka AB,29.277679
20150216,Krka AB,26.837500
20150302,Krka AB,21.958036
20150316,Krka AB,20.006250
20150330,Krka AB,17.078571
...,...,...
20200127,Krka AB,0.579464
20200210,Krka AB,0.715500
20200224,Krka AB,0.715500


In [245]:
plt.scatter(e.min, 'Date')
plt.show

TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
medprices_dk_copy = medprices_dk.copy()
lamotrigin=medprices_dk_copy.loc[medprices_dk_copy['Medicine'].isin(['Lamotrigin "1A Farma"', 'Lamotrigin "Stada"', 'Lamotrigin "Orifarm"','Lamotrigin "Bluefish"','Lamotrigin "Aurobindo"', 'Lamotrigin "Amneal"', 'Lamotrigin "Abacus Medicine"', 'Lamotrigin "Mylan"']) ]
I=lamotrigin.Indicator.str.contains('AUP_pr_DDD')
lamotrigin=lamotrigin.loc[I, :]
drop_list=['ATC', 'Form', 'Medicine']
lamotrigin.drop(drop_list, axis=1, inplace=True)
lamotrigin.reset_index()


In [ ]:
# 4. Create long format of dataframe 

lamotrigin_long= pd.wide_to_long(lamotrigin, stubnames='e', i="Product_number", j='Date') # Created long format of the lamotrigin Dataframe
e_grouped = lamotrigin_long.groupby('Product_number')['e'] # Grouped by product number
lamotrigin_long= lamotrigin_long.sort_values(['Product_number','Date']) # Organized product number by date
lamotrigin_long

Average price per daily dose

In [ ]:
# Calculated mean value of each product number 
e_mean = lamotrigin_long.groupby('Product_number')['e'].mean() 
e_mean.name ='e_mean'
lamotrigin_new = lamotrigin.set_index('Product_number').join(e_mean, how='left') #merged mean values into original wide dataframe

# Plot the distribution of price per daily dose in lamotrigin_new
plt.hist(lamotrigin_new['e_mean'], bins=50, align='left', color='b', edgecolor='red',
              linewidth=1)
 
# Add axis labels
plt.xlabel("Price DKK/daily dose")
plt.ylabel("Frequency")
plt.title("Distribution of the avarage price per daily dose")
plt.axvline(lamotrigin_new['e_mean'].mean(), color='red', linestyle='dashed', linewidth=1)

plt.show()

In [ ]:
#ax = sns.scatterplot(x='e', y='Date')


In [ ]:
Date = pd.DataFrame(columns=['Date'])
e = pd.DataFrame(columns=['e'])
plt.scatter(date, e)
plt.xlabel("Date")
plt.ylabel("Price")
plt.show

## 5. Conclusion

Even though it can not be visualised, the prices behaves as price cycles. Whether or not there is tacit collusion in the medicin market, can only be speculated, and not proven in this assignment. However the data has been cleaned, and structured, and therefore ready to be analyzed.